## Qualifying Fact Analysis

#### Import

In [1]:
%matplotlib inline
import psycopg2
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import pip
import geopandas
import altair as alt
from descartes.patch import PolygonPatch
from datetime import datetime
from apyori import apriori 
import plotly.express as px
from sklearn.svm import SVC
import warnings
warnings.simplefilter("ignore")

# importing ML libraries 
from sklearn.preprocessing import LabelEncoder
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error, mean_squared_error, median_absolute_error, r2_score
import pickle


### Data  Extraction

#### Connection to database 

In [2]:
def DbConnect():
    conn = psycopg2.connect(host="localhost",database="FormulaOne",port=5432,user='postgres',password='root')
    return conn

In [3]:
circuit = pd.read_sql('select * from "CircuitDim"', con=DbConnect())
races= pd.read_sql('select * from "RaceDim"', con=DbConnect())
date= pd.read_sql('select * from "DateTimeDim"', con=DbConnect())
races= pd.read_sql('select * from "RaceDim"', con=DbConnect())
driver= pd.read_sql('select * from "DriverDim"', con=DbConnect())
constructor= pd.read_sql('select * from "ConstructorsDim"', con=DbConnect())
status= pd.read_sql('select * from "StatusDim"', con=DbConnect())
result= pd.read_sql('select * from "RaceResultFact"', con=DbConnect())

### Merging to a single DataFrame

##### Renaming Keys

In [4]:
races.rename(columns = {'datetime_fk':'dateId'}, inplace = True)
races.rename(columns = {'circuit_fk':'CircuitId'}, inplace = True)
result.rename(columns = {'race_fk':'raceId'}, inplace = True)
result.rename(columns = {'driver_fk':'DriverID'}, inplace = True)
result.rename(columns = {'constructor_fk':'constructorId'}, inplace = True)
result.rename(columns = {'status_fk':'statusId'}, inplace = True)

In [5]:
df1 = pd.merge(races,date, on='dateId', how='inner')
Racedim = pd.merge(df1,circuit, on='CircuitId', how='inner')
df2 = pd.merge(result,Racedim, on='raceId', how='inner')
df3 = pd.merge(df2,driver, on='DriverID', how='inner')
df4 = pd.merge(df3,constructor, on='constructorId', how='inner')
df5 = pd.merge(df4,status, on='statusId', how='inner')

### Data Anlysis

##### Preparing

In [6]:
Fact=df5

In [7]:
Fact.columns

Index(['raceId', 'DriverID', 'constructorId', 'statusId', 'points_x', 'rank',
       'laps', 'fastest_lapspeed', 'wins', 'laptime', 'pitstop', 'penalty_fk',
       'round', 'name_x', 'CircuitId', 'dateId', 'weather_fk', 'date', 'day',
       'month', 'year', 'time', 'circuitRef', 'name_y', 'location', 'latitude',
       'longitude', 'Altitude', 'country', 'img', 'DriverRef', 'Number',
       'Code', 'ForeName', 'SurName', 'Dob', 'Nationality', 'points_y', 'car',
       'constructorRef', 'constructorName', 'constructorNationality',
       'status'],
      dtype='object')

##### Deleting data before 2014

Due to the lack of data accuracy before 2014 and the constant changes occurring on the formula one rules we saw best to only consider data from 2014 and above.

In [8]:
Fact=Fact.drop(Fact[(Fact["year"] <=2014)].index)

##### Cleaning

In [9]:
Fact["nameDriver"]=Fact["ForeName"]+" "+Fact["SurName"]
Fact["nameDriver"]
Fact['Dob']=pd.to_datetime(Fact['Dob'])
date=datetime.today()-Fact['Dob']
Fact['age']=round(date.dt.days/365)

Calculating the drivers' age is more significant that having a birth date as it provides us a mesurable value.

In [10]:
Fact.drop(columns=['raceId',
                   'DriverID',
                   'constructorId',
                   'CircuitId',
                   'dateId',
                   'constructorRef',
                   'DriverRef',
                   'circuitRef',
                   'ForeName',
                   'SurName',
                   'Dob',
                   'Number',
                    'day',
                   'month'],axis=1,inplace=True)

In [11]:
Fact.rename(columns = {'name_x':'nameGP','name_y':'nameCircuit'}, inplace = True)

In [12]:
Fact.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2390 entries, 70 to 3542
Data columns (total 31 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   statusId                2390 non-null   int64  
 1   points_x                2390 non-null   float64
 2   rank                    2390 non-null   int64  
 3   laps                    2390 non-null   int64  
 4   fastest_lapspeed        2381 non-null   float64
 5   wins                    2390 non-null   int64  
 6   laptime                 2390 non-null   object 
 7   pitstop                 2390 non-null   object 
 8   penalty_fk              2009 non-null   float64
 9   round                   2390 non-null   int64  
 10  nameGP                  2390 non-null   object 
 11  weather_fk              2390 non-null   int64  
 12  date                    2390 non-null   object 
 13  year                    2390 non-null   int64  
 14  time                    2390 non-null  

In [13]:
Fact.shape

(2390, 31)

In [14]:
Fact.head()

,statusId,points_x,rank,laps,fastest_lapspeed,wins,laptime,pitstop,penalty_fk,round,...,img,Code,Nationality,points_y,car,constructorName,constructorNationality,status,nameDriver,age
70,1,10.0,5,56,207.334,0,01:32:55.944000,00:01:12.289000,NaN,1,...,https://f1chronicle.com/wp-content/uploads/202...,PER,Mexican,190.0,Red Bull Racing Honda,Red Bull,Austrian,Finished,Sergio Pérez,32.0
71,1,0.0,11,63,222.585,0,02:03:41.749000,00:27:05.766000,NaN,2,...,https://f1chronicle.com/wp-content/uploads/202...,PER,Mexican,190.0,Red Bull Racing Honda,Red Bull,Austrian,Finished,Sergio Pérez,32.0
72,1,12.0,4,66,207.715,0,01:35:11.156000,00:01:10.597000,NaN,3,...,https://f1chronicle.com/wp-content/uploads/202...,PER,Mexican,190.0,Red Bull Racing Honda,Red Bull,Austrian,Finished,Sergio Pérez,32.0
73,1,10.0,5,66,211.743,0,01:34:11.351000,00:00:43.727000,NaN,4,...,https://f1chronicle.com/wp-content/uploads/202...,PER,Mexican,190.0,Red Bull Racing Honda,Red Bull,Austrian,Finished,Sergio Pérez,32.0
74,1,12.0,4,78,161.138,0,01:39:17.310000,00:00:24.548000,NaN,5,...,https://f1chronicle.com/wp-content/uploads/202...,PER,Mexican,190.0,Red Bull Racing Honda,Red Bull,Austrian,Finished,Sergio Pérez,32.0


In [15]:
Fact.tail()

,statusId,points_x,rank,laps,fastest_lapspeed,wins,laptime,pitstop,penalty_fk,round,...,img,Code,Nationality,points_y,car,constructorName,constructorNationality,status,nameDriver,age
3536,14,0.0,18,67,197.316,0,01:31:33.724000,00:00:52.198000,NaN,18,...,https://www.autocar.co.uk/sites/autocar.co.uk/...,RSS,American,NaN,None,Manor Marussia,British,+4 Laps,Alexander Rossi,31.0
3537,14,0.0,15,74,150.413,0,01:59:41.780000,00:01:22.789000,142.0,6,...,https://f1chronicle.com/wp-content/uploads/202...,HAR,Indonesian,4.0,Scuderia Toro Rosso Honda,Manor Marussia,British,+4 Laps,Rio Haryanto,29.0
3540,137,0.0,19,28,212.478,0,00:43:19.322000,00:00:33.027000,44.0,1,...,https://f1chronicle.com/wp-content/uploads/202...,RIC,Australian,115.0,McLaren Mercedes,Renault,French,Damage,Daniel Ricciardo,33.0
3541,60,0.0,18,53,201.487,0,01:27:16.482000,00:00:25.676000,44.0,2,...,https://f1chronicle.com/wp-content/uploads/202...,RIC,Australian,115.0,McLaren Mercedes,Renault,French,Out of fuel,Daniel Ricciardo,33.0
3542,139,0.0,18,47,222.930,0,01:04:14.971000,00:00:30.708000,104.0,13,...,https://f1chronicle.com/wp-content/uploads/202...,MAG,Danish,1.0,Haas Ferrari,Haas F1 Team,American,Illness,Kevin Magnussen,30.0


In [16]:
Fact.columns

Index(['statusId', 'points_x', 'rank', 'laps', 'fastest_lapspeed', 'wins',
       'laptime', 'pitstop', 'penalty_fk', 'round', 'nameGP', 'weather_fk',
       'date', 'year', 'time', 'nameCircuit', 'location', 'latitude',
       'longitude', 'Altitude', 'country', 'img', 'Code', 'Nationality',
       'points_y', 'car', 'constructorName', 'constructorNationality',
       'status', 'nameDriver', 'age'],
      dtype='object')

In [17]:
Fact["rank"].unique()

array([ 5, 11,  4,  1,  3,  6,  9,  2, 10,  7,  8, 14, 15, 12, 13, 16, 19,
       18, 17, 20, 22, 21], dtype=int64)

In [18]:
selected_columns = Fact[["Code","rank","fastest_lapspeed","age","Nationality","points_y","year"]]
df = selected_columns.copy()
df

,Code,rank,fastest_lapspeed,age,Nationality,points_y,year
70,PER,5,207.334,32.0,Mexican,190.0,2021
71,PER,11,222.585,32.0,Mexican,190.0,2021
72,PER,4,207.715,32.0,Mexican,190.0,2021
73,PER,5,211.743,32.0,Mexican,190.0,2021
74,PER,4,161.138,32.0,Mexican,190.0,2021
...,...,...,...,...,...,...,...
3536,RSS,18,197.316,31.0,American,NaN,2015
3537,HAR,15,150.413,29.0,Indonesian,4.0,2016
3540,RIC,19,212.478,33.0,Australian,115.0,2019
3541,RIC,18,201.487,33.0,Australian,115.0,2019


In [19]:
df['winning']=0
for i in range(0,len(df)):
    pos=df['rank'].iat[i]
    if pos>0 and pos<10:
         df['winning'].iat[i] = int(1)
    else:
        
         df['winning'].iat[i] = int(0)

In [20]:
df.drop(columns=['rank'],axis=1,inplace=True)

In [21]:
df1=df
df1=df1.groupby(["Code"]).sum('winning')

In [22]:
df1["performance"]=df1["winning"]/Fact.groupby(["Code"]).size()

In [23]:
df=df1["performance"].to_frame()


In [24]:
df1 = pd.merge(df,driver, on='Code', how='inner')
df1['Dob']=pd.to_datetime(df1['Dob'])
date=datetime.today()-df1['Dob']
df1['age']=round(date.dt.days/365)
df1.drop(columns=['DriverID','DriverRef','Number','ForeName','SurName','Dob','car'],axis=1,inplace=True)
df1['points'] = df1['points'].fillna(0).astype(np.int64, errors='ignore')
df1['age'] = df1['age'].fillna(0).astype(np.int64, errors='ignore')
df=df1
df

,Code,performance,Nationality,points,age
0,AIT,0.000000,British,0,27
1,ALB,0.578947,Thai,105,26
2,ALO,0.307692,Spanish,81,41
3,BOT,0.829268,Finnish,226,33
4,BUT,0.285714,British,21,42
5,ERI,0.068493,Swedish,9,32
6,FIT,0.000000,Brazilian,0,26
7,GAS,0.492063,French,110,26
8,GIO,0.065217,Italian,3,28
9,GRO,0.247619,French,2,36


In [25]:
df.sort_values(by="performance",ascending=False).head(15)

,Code,performance,Nationality,points,age
32,ROS,0.973684,German,385,37
11,HAM,0.968000,British,387,37
3,BOT,0.829268,Finnish,226,33
41,VER,0.808696,French,395,32
42,VER,0.808696,Dutch,395,25
43,VET,0.788136,German,43,35
31,RIC,0.711864,Australian,115,33
26,NOR,0.666667,British,160,22
18,LEC,0.640625,Monegasque,159,25
29,PER,0.639344,Mexican,190,32


In [26]:
le = LabelEncoder()
df["Code"] = le.fit_transform(df["Code"])
df["Nationality"] = le.fit_transform(df["Nationality"])

In [27]:
df.sort_values(by="performance",ascending=False).head(15)

,Code,performance,Nationality,points,age
32,31,0.973684,10,385,37
11,11,0.968000,4,387,37
3,3,0.829268,8,226,33
41,40,0.808696,9,395,32
42,40,0.808696,7,395,25
43,41,0.788136,10,43,35
31,30,0.711864,1,115,33
26,25,0.666667,4,160,22
18,17,0.640625,15,159,25
29,28,0.639344,14,190,32


In [28]:
X = df.drop(['performance'], axis=1)
y = df[['performance']]
X_train, X_test, y_train, y_test=train_test_split(X, y, test_size=0.2, random_state=3)

In [29]:
regressor = LinearRegression()

#Fitting model with trainig data
regressor.fit(X, y)

LinearRegression()

In [30]:
test={'Code':['11'],'Nationality':['24'],'points':['170'],'age':['30']}
df=pd.DataFrame(test)
pred = regressor.predict(df)
print(pred)

[[0.50502897]]


In [31]:
import pickle
pickle.dump(regressor, open('model.pkl','wb'))

# Loading model to compare the results
model = pickle.load(open('model.pkl','rb'))